In [2]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import pandas as pd
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from tqdm import tqdm
import random

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns

path_root = './'
path_data = './'
path_submissions = '/'

target_name = 'target'
scores_folds = {}

DEBUG = False

In [186]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))


def get_stock_stat(stock_id : int, dataType = 'train'):
    key = ['stock_id', 'time_id', 'seconds_in_bucket']
    
    #Book features
    df_book = pd.read_parquet(os.path.join(path_data, 'book_{}.parquet/stock_id={}/'.format(dataType, stock_id)))
    df_book['stock_id'] = stock_id
    cols = key + [col for col in df_book.columns if col not in key]
    df_book = df_book[cols]
    
    df_book['wap1'] = (df_book['bid_price1'] * df_book['ask_size1'] +
                                    df_book['ask_price1'] * df_book['bid_size1']) / (df_book['bid_size1'] + df_book['ask_size1'])
    df_book['wap2'] = (df_book['bid_price2'] * df_book['ask_size2'] +
                                    df_book['ask_price2'] * df_book['bid_size2']) / (df_book['bid_size2'] + df_book['ask_size2'])
    df_book['log_return1'] = df_book.groupby(by = ['time_id'])['wap1'].apply(log_return).fillna(0)
    df_book['log_return2'] = df_book.groupby(by = ['time_id'])['wap2'].apply(log_return).fillna(0)
    
    features_to_apply_realized_volatility = ['log_return'+str(i+1) for i in range(2)]
    stock_stat = df_book.groupby(by = ['stock_id', 'time_id'])[features_to_apply_realized_volatility]\
                        .agg(realized_volatility).reset_index()

    #Trade features
    trade_stat =  pd.read_parquet(os.path.join(path_data,'trade_{}.parquet/stock_id={}'.format(dataType, stock_id)))
    trade_stat = trade_stat.sort_values(by=['time_id', 'seconds_in_bucket']).reset_index(drop=True)
    trade_stat['stock_id'] = stock_id
    cols = key + [col for col in trade_stat.columns if col not in key]
    trade_stat = trade_stat[cols]
    trade_stat['trade_log_return1'] = trade_stat.groupby(by = ['time_id'])['price'].apply(log_return).fillna(0)
    trade_stat = trade_stat.groupby(by = ['stock_id', 'time_id'])[['trade_log_return1']]\
                           .agg(realized_volatility).reset_index()
    #Joining book and trade features
    stock_stat = stock_stat.merge(trade_stat, on=['stock_id', 'time_id'], how='left').fillna(-999)
    
    return stock_stat

def get_dataSet(stock_ids : list, dataType = 'train'):
    
    if DEBUG and dataType == 'train':
        stock_ids = stock_ids[:10]

    stock_stat = Parallel(n_jobs=-1)(
        delayed(get_stock_stat)(stock_id, dataType) 
        for stock_id in stock_ids
    )
    
    stock_stat_df = pd.concat(stock_stat, ignore_index = True)

    return stock_stat_df

def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

In [187]:
train = pd.read_csv(os.path.join(path_data,'train.csv'))

In [188]:
test = pd.read_csv(os.path.join(path_data, 'test.csv'))

In [189]:
train = pd.read_csv(os.path.join(path_data, 'train.csv'))
%time train_stock_stat_df = get_dataSet(stock_ids = train['stock_id'].unique(), dataType = 'train')
train = pd.merge(train, train_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left').fillna(0)
print('Train shape: {}'.format(train.shape))
display(train.head(2))

test = pd.read_csv(os.path.join(path_data, 'test.csv'))
test_stock_stat_df = get_dataSet(stock_ids = test['stock_id'].unique(), dataType = 'test')
test = pd.merge(test, test_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left').fillna(0)
print('Test shape: {}'.format(test.shape))
display(test.head())

Wall time: 2min 7s
Train shape: (428932, 6)


,stock_id,time_id,target,log_return1,log_return2,trade_log_return1
0,0,5,0.004136,0.004499,0.006999,0.002006
1,0,11,0.001445,0.001204,0.002476,0.000901


Test shape: (3, 6)


,stock_id,time_id,row_id,log_return1,log_return2,trade_log_return1
0,0,4,0-4,0.000294,0.000252,0.000295
1,0,32,0-32,0.000000,0.000000,0.000000
2,0,34,0-34,0.000000,0.000000,0.000000


In [10]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

In [11]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [164]:
#define dataset/dataloader

class VolatilityDataset(Dataset):
    def __init__(self, X,y, emb_cols=['stock_id','time_id']):
        X = X.copy()
        self.X1 = X.loc[:,emb_cols].copy().values.astype(np.int8)
        self.X2 = X.drop(columns=emb_cols).copy().values.astype(np.float32)
        
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (torch.tensor(np.expand_dims(self.X1[idx], 0)).int().to(device), torch.FloatTensor(np.expand_dims(self.X2[idx],0)).to(device)), torch.FloatTensor(np.expand_dims(self.y[idx],0)).to(device)
    
class VolatilityDatasetTest(Dataset):
    def __init__(self, X, emb_cols=['stock_id','time_id']):
        X = X.copy()
        
        self.X1 = X.loc[:,emb_cols].copy().values.astype(np.int8)
        self.X2 = X.drop(columns=emb_cols).copy().values.astype(np.float32)
        
    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, idx):
        return (torch.tensor(np.expand_dims(self.X1[idx],0)).int().to(device),torch.FloatTensor(np.expand_dims(self.X2[idx],0)).to(device)

In [190]:
train['stock_id'] = train['stock_id'].astype(np.int8)

In [191]:
print(train.shape)
display(train.head())

(428932, 6)


,stock_id,time_id,target,log_return1,log_return2,trade_log_return1
0,0,5,0.004136,0.004499,0.006999,0.002006
1,0,11,0.001445,0.001204,0.002476,0.000901
2,0,16,0.002168,0.002369,0.004801,0.001961
3,0,31,0.002195,0.002574,0.003637,0.001561
4,0,62,0.001747,0.001894,0.003257,0.000871


In [193]:
train_dataset = VolatilityDataset(train.drop(['target','time_id'], axis=1), train['target'], emb_cols=['stock_id'])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

for (emb, cont), target in train_loader:
#     emb, cont, target = emb.unsqueeze(0), cont.unsqueeze(0), target.unsqueeze(0)
    print((emb, cont), target)
    print((emb.shape, cont.shape), target.shape)
    break

(tensor([[[116]],

        [[ 41]],

        [[ 74]],

        [[124]]], device='cuda:0', dtype=torch.int32), tensor([[[0.0036, 0.0056, 0.0026]],

        [[0.0010, 0.0011, 0.0007]],

        [[0.0008, 0.0013, 0.0009]],

        [[0.0020, 0.0026, 0.0017]]], device='cuda:0')) tensor([[0.0056],
        [0.0014],
        [0.0012],
        [0.0025]], device='cuda:0')
(torch.Size([4, 1, 1]), torch.Size([4, 1, 3])) torch.Size([4, 1])


In [211]:
class Encoder(nn.Module):
    
    def __init__(self, n_features, embedding_size=16, num_embeddings=max(train['stock_id']) +1):
        super(Encoder, self).__init__()
        
        self.emb = nn.Embedding(num_embeddings, embedding_size)
        self.emb_drop = nn.Dropout(0.25)
        
        self.bn1 = nn.BatchNorm1d(1)
        
        self.n_features = n_features
        
        self.embedding_size, self.hidden_dim = embedding_size, 2*embedding_size
        
        self.rnn1 = nn.LSTM(
            input_size = n_features,
            hidden_size = self.hidden_dim,
            num_layers = 1,
            batch_first = True
        )
        
        self.rnn2 = nn.LSTM(
            input_size = self.hidden_dim,
            hidden_size = embedding_size,
            num_layers = 1,
            batch_first = True
        )
        
        
    def forward(self, x_cat, x_cont):
        x1 = self.emb(x_cat)
        x1 = torch.flatten(x1, end_dim=1)
        x1 = self.emb_drop(x1)
        
        x2 = self.bn1(x_cont)
        x = torch.cat([x1,x2],1)
        x, (_,_) = self.rnn1(x)
        x = F.relu(x)
        x, (hidden,_) = self.rnn2(x)
        
        return hidden
        
        

In [171]:
class Decoder(nn.Module):
    
    def __init__(self, seq_len, input_dim=16, n_features=10):
        super(Decoder, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim, self.n_features = 2*input_dim, n_features
        
        self.rnn1 = nn.LSTM(
            input_size = input_dim,
            hidden_size = input_dim,
            num_layers = 1,
            batch_first = True
        )
        
        self.rnn2 = nn.LSTM(
            input_size = input_dim,
            hidden_size = self.hidden_dim,
            num_layers = 1,
            batch_first = True
        )
        
        self.output_layer = nn.Linear(self.hidden_dim, n_features)
        
    def forward(self, x, _):
        x, (hidden, cell) = self.rnn1(x)
        x, (hidden, cell) = self.rnn2(x)
        
        return self.output_layer(x)
        
        

In [195]:
device = torch.device('cuda')

In [214]:
class RAE(nn.Module):
    
    def __init__(self, n_features, embedding_dim=3):
        super(RAE, self).__init__()
        
        self.encoder = Encoder(n_features, embedding_dim).to(device)
        
        self.decoder = Decoder(embedding_dim, n_features).to(device)
        
    def forward(self, cats, conts):
        x = self.encoder(cats, conts)
        x = self.decoder(x,conts)
        
        return torch.sigmoid(x)

In [206]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 ))

In [207]:
def prepare_dataset(train, valid, test=None, batch_size=64, drop_cols=['target', 'time_id'], emb_cols=['stock_id']):
    train_dataset = VolatilityDataset(train.drop(drop_cols, axis=1), train['target'], emb_cols=emb_cols)
    valid_dataset = VolatilityDataset(valid.drop(drop_cols, axis=1), valid['target'], emb_cols=emb_cols)    
    
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    
    return train_dl, valid_dl

In [208]:
kf = model_selection.KFold(n_splits=4, shuffle=True, random_state=42)
seed_everything(46)

In [215]:
def train_epoch(train_dl, valid_dl, model, loss_fn, opt, sch, epoch, fold, device=device):
    # taining loop
    model.train()
    running_loss_ = 0
    
    pbar = tqdm(enumerate(train_dl), total=len(train_dl))
    for i, ((cats, counts), targets) in pbar:
        cats, counts, targets = cats.to(device), counts.to(device), targets.unsqueeze(1).to(device)
        
        opt.zero_grad()
        y_pred = model(cats, counts)
        loss = loss_fn(y_pred.float(), targets.float())
        
        loss.backward()
        opt.step()
        
        running_loss_ += loss.item()
        if (i+1) % 100 == 0:
            pbar.set_description(f"running loss:{running_loss_ / (i+1): 0.6f}")
    
    sch.step(loss)

    epoch_loss = running_loss_ / len(train_dl)
    #print(f'==> Epoch {epoch} TRAIN loss: {epoch_loss:.6f}')
    
    # Validation loop
    model.eval()
    valid_loss = 0
    best_loss = np.inf
    
    for i, ((cats, counts), targets) in enumerate(valid_dl):
        cats, counts, targets = cats.to(device), counts.to(device), targets.unsqueeze(1).to(device)
        
        with torch.no_grad():
            y_pred = model(cats, counts)
            val_loss = loss_fn(y_pred.float(), targets.float())
            
        valid_loss += val_loss.item() * targets.shape[0]
    sch.step(valid_loss)
    
    valid_epoch_loss = valid_loss / len(valid_dl)
    print(f'==>F{fold}, Epoch {epoch} VALID loss: {valid_epoch_loss:.8f}')
    
    if valid_epoch_loss < best_loss:
        best_loss = valid_epoch_loss
        torch.save(model.state_dict(), f'FOLD{fold}_optive_model.pth')
    
    model.train()
    return model, epoch_loss, valid_epoch_loss

In [216]:
epochs = 10
for _fold, (tr, te) in enumerate(kf.split(range(len(train)))):
    print(_fold)
    
    trainSet = train.loc[tr].reset_index(drop=True)
    valSet = train.loc[te].reset_index(drop=True)
    
    trainDl, valDl = prepare_dataset(trainSet, valSet)
    
    model = RAE(3).to(device)
    
    loss_fn = RMSELoss
    
    opt = optim.Adam(model.parameters(), lr=0.01)
    sch = optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.2, patience=3)
    
    counter = 0 
    for epoch in range(epochs):
        model, epoch_loss, valid_epoch_loss = train_epoch(trainDl, valDl, model, loss_fn, opt, sch, epoch, _fold, device=device)
        
        

  0%|                                                                                 | 3/5027 [00:00<03:00, 27.79it/s]

0


  0%|                                                                                 | 4/5027 [00:00<02:09, 38.84it/s]

==>F0, Epoch 0 VALID loss: 0.19185096


  0%|                                                                                 | 4/5027 [00:00<02:05, 40.00it/s]

...

==>F3, Epoch 7 VALID loss: 0.18260365


  0%|                                                                                 | 2/5027 [00:00<04:33, 18.35it/s]

==>F3, Epoch 8 VALID loss: 0.18256183


running loss: 0.002863: 100%|██████████████████████████████████████████████████████| 5027/5027 [02:58<00:00, 28.09it/s]


==>F3, Epoch 9 VALID loss: 0.18262851
